<a href="https://colab.research.google.com/github/pstakuu/me/blob/master/ProcessExcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openpyxl
from openpyxl import Workbook, load_workbook
import xlsxwriter
from xlsxwriter.utility import xl_col_to_name

Inserted import xlsxwriter for cell_col_to_name() because I was trying to figure out how to map A-1, B-2, C-3, so that I could do dynamic column references(if you ever had more than 4 (information/naviga/commer/transac..)) based on the max_column. In a forum: https://stackoverflow.com/a/45177642 in the comment on this comment, someone posted in 2019 that there was already functions in the xlsxwriter.utility for doing this.. https://xlsxwriter.readthedocs.io/working_with_cell_notation.html#cell-utility

In [ ]:
wb = load_workbook(r'C:\users\bboyd\downloads\finalAGAIN.xlsx')

In [ ]:
ws = wb['Total Volumes']

ws['C1'] = "Informational"
ws['D1'] = "Navigational"
ws['E1'] = "Commercial"
ws['F1'] = "Transactional"

There was a TYPO from the original :) 

ws['E1'] = "Transactional"

becomes

ws['F1'] = "Transactional"

In [ ]:
#ws['C2'] = r'=COUNTIFS(Sheet1!A:A, $A2, Sheet1!D:D, "*"  & $D$1 & "*")'
#reference from your script

In [ ]:
lastRow = ws.max_row
#i end up dropping these variables and just reference ws.max_row


https://openpyxl.readthedocs.io/en/stable/api/openpyxl.worksheet.worksheet.html#openpyxl.worksheet.worksheet.Worksheet.max_row

In [ ]:
for i in range(2,lastRow + 1):
  #print(i)
  keyword = ws.cell(row=i,column=1)
  #print(keyword.value)

  equation = f'=COUNTIFS(Sheet1!A:A, "{keyword.value}", Sheet1!D:D, "*"  & $D$1 & "*")'
  #print(equation)
  ws.cell(row=i,column=3).value = equation

In [ ]:
#wb.save(r'C:\users\bboyd\downloads\finaltest.xlsx')

There were two issues with the equation you had:

1 -- this line returns a cell object:  keyword = ws.cell(row=i,column=1)

    https://openpyxl.readthedocs.io/en/stable/api/openpyxl.worksheet.worksheet.html#openpyxl.worksheet.worksheet.Worksheet.cell
    Return type:	
    openpyxl.cell.cell.Cell
    
The cell object has a values property:   #https://openpyxl.readthedocs.io/en/stable/api/openpyxl.cell.cell.html#openpyxl.cell.cell.Cell.value3

I found that by inserting the print(i), print(keyword), and print(equation) lines into the function.  As soon as I did that I saw a <Cell > that was being inserted and that was breaking the Excel sheet.
    
I then replaced the print(keyword) with print(keyword.value) to make sure it was pulling the right data.
    
Then i updated the equation with {keyword.value} instead of {keyword}

    
2 -- The equation string needed two adjustments
    If you use single quotes, you are declaring the string as a 'string literal', which means "python, don't interpolate this", i.e., don't substitute variables for what you think are varibles in here.
    
We used the r'' for raw string literal so that we didn't have to escape all the special characters (" * $ & ) etc..so that by the time python got done with the string, Excel would see it right.. but now we are introducing variables into it... 
    
soo...
f'' string for Python Literal String Interpolation: 
https://www.geeksforgeeks.org/python-string-interpolation/

And then with that it worked! But not fully... I had to add double quotes around the variable so that the quotes were written into the formula: "{keyword.value}".
    
    

In [ ]:
lastCol = ws.max_column

In [ ]:
for j in range(3,lastCol+1):
    print(j)

3
4
5
6


In [ ]:
lastCol
#i end up dropping these variables and just reference ws.max_column

6

In [ ]:
for j in range(3,lastCol+1):
    print(j)
    currentColName = xl_col_to_name(j)
    print (currentColName)
    

3
D
4
E
5
F
6
G


Testing the output


In [ ]:
for j in range(3,lastCol+1):
   # print(j)
    currentColName = xl_col_to_name(j - 1)
   # print (currentColName)
    for i in range(2,lastRow + 1):
      #print(i)
      keyword = ws.cell(row=i,column=1)
      #print(keyword.value)
      
      equation = f'=COUNTIFS(Sheet1!A:A, "{keyword.value}", Sheet1!D:D, "*"  & ${currentColName}$1 & "*")'
      #print(equation)
      ws.cell(row=i,column=j).value = equation

So now I've introduced the extra J loop to go through columns, and use the currentColName to map the column # to column letter, so that I can reference that in the equation.  Going from $D$1 to ${currentColName}$1.
The last thing to do is to update the last cell update to write the function, and use the current value of j instead of hardcoded column #

In [ ]:
wb.save(r'C:\users\bboyd\downloads\finaltest.xlsx')

In [ ]:
import openpyxl
from openpyxl import Workbook, load_workbook
import xlsxwriter
from xlsxwriter.utility import xl_col_to_name

wb = load_workbook(r'C:\users\bboyd\downloads\finaltest.xlsx')
ws = wb['Total Volumes']

ws['C1'] = "Informational"
ws['D1'] = "Navigational"
ws['E1'] = "Commercial"
ws['F1'] = "Transactional"


for j in range(3,ws.max_column + 1):
    currentColName = xl_col_to_name(j -1 )
    for i in range(2,ws.max_row + 1):
      keyword = ws.cell(row=i,column=1)      
      equation = f'=COUNTIFS(Sheet1!A:A, "{keyword.value}", Sheet1!D:D, "*"  & ${currentColName}$1 & "*")'
      ws.cell(row=i,column=j).value = equation

wb.save(r'C:\users\bboyd\downloads\finalAGAIN.xlsx')

I realized I needed to subtract 1 from j in converting the currentColName from the # since it's 0 base